In [ ]:
#Plain Ridge regression 

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

def MSE(y_data,y_model):
    n = np.size(y_model)
    return np.sum((y_data-y_model)**2)/n

def R2(y_data, y_model):
    return 1 - np.sum((y_data - y_model) ** 2) / np.sum((y_data - np.mean(y_data)) ** 2)

#grid size
n = 200 

# Make data set.
x = np.linspace(0,(np.pi)/2,n).reshape(-1, 1)

#Northern H. parameters
s0 = 1
s2 = -0.473
a0 = 0.675
a2 = -0.192
i2 = -0.165

#flux function (eqn. (14) from Stone_1978)
y = 0.5*(s0*a2+s2*a0+(2/7)*s2*a2-i2)*((np.sin(x))**3-np.sin(x))

#noisy flux function
y_noisy = np.random.normal(y, abs(y*0.05)) 

maxdegree=10
MSE_difference = np.zeros((maxdegree, nlambdas))

for degree in range(maxdegree):
    #polynomial fit
    poly = PolynomialFeatures(degree=degree)
    X = poly.fit_transform(x)

    # Repeat now for Ridge regression and various values of the regularization parameter, λ
    I = np.eye(1,1)

    # Decide which values of lambda to use
    nlambdas = 15

    MSEPredict = np.zeros(nlambdas)
    MSEfit = np.zeros(nlambdas)
    
    R2Predict = np.zeros(nlambdas)
    R2fit = np.zeros(nlambdas)   
    
    
    lambdas = np.logspace(-4, 10, nlambdas)
    
    for i in range(nlambdas):
        
        lmb = lambdas[i]
        
        #splitting of the data
        X_train, X_test, y_train, y_test = train_test_split(X, y_noisy, test_size=0.25, random_state=42)     

        Ridgebeta = np.linalg.inv(X_train.T @ X_train+lmb*I) @ X_train.T @ y_train

        # Prediction
        ypredict = X_test @ Ridgebeta
        yfit = X_train @ Ridgebeta
       
        MSEPredict[i]=MSE(y_test,ypredict)
        #print('predict', MSEPredict)
        MSEfit[i]=MSE(y_train,yfit) 
        #print('fit', MSEfit)
        R2Predict[i]= R2(y_test,ypredict)
        R2fit[i]=R2(y_train,yfit)

        MSE_difference[degree, i]= np.abs(MSEPredict[i]-MSEfit[i])
        #print(MSE_difference)
        
        if i==5 and degree==6:
            print(Ridgebeta)

    if degree in [1, 6, 9]:    
        #I'm plotting the prediction
        print(degree)
        plt.figure()
        #plt.ylim([0, 0.00125])
        plt.plot(np.log10(lambdas), MSEPredict, label = 'MSE test Ridge')
        plt.plot(np.log10(lambdas), MSEfit, label = 'MSE train Ridge')
        plt.xlabel('log10(lambda)')
        plt.ylabel('MSE')
        plt.legend()
        plt.show()

#    plt.figure()
#    plt.plot(np.log10(lambdas), R2Predict, label = 'R2 test Ridge')
#    plt.plot(np.log10(lambdas), R2fit, label = 'R2 train Ridge')
#    plt.xlabel('log10(lambda)')
#    plt.ylabel('MSE')
#    plt.legend()
#    plt.show()

#print(MSE_difference)

pd.DataFrame(MSE_difference)
mse_difference = pd.DataFrame(MSE_difference)

plt.figure(figsize=(15,10))
sns.heatmap(data=mse_difference, annot=True,  fmt="1.1e", cmap="crest")
plt.xlabel("lambda number")
plt.ylabel("pol.degree")
plt.show()

In [ ]:
#Bootstrap resampling on Ridge regression

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import resample
from sklearn.linear_model import Ridge
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

def MSE(y_data,y_model):
    n = np.size(y_model)
    return np.sum((y_data-y_model)**2)/n

#grid size
n = 200 

# Make data set.
x = np.linspace(0,(np.pi)/2,n).reshape(-1, 1)

#Northern H. parameters
s0 = 1
s2 = -0.473
a0 = 0.675
a2 = -0.192
i2 = -0.165

#flux function (eqn. (14) from Stone_1978)
y = 0.5*(s0*a2+s2*a0+(2/7)*s2*a2-i2)*((np.sin(x))**3-np.sin(x))

#noisy flux function
y_noisy = np.random.normal(y, abs(y*0.05)) 

MSE_Bootstrap_compare_d1=np.zeros(nlambdas)
MSE_Bootstrap_compare_d6=np.zeros(nlambdas)
MSE_Bootstrap_compare_d9=np.zeros(nlambdas)

for degree in [1, 6, 9]:

    #Polynomial fit but now for degree=ct.=6
    poly = PolynomialFeatures(degree=degree)
    X = poly.fit_transform(x)

    #splitting of the data
    X_train, X_test, y_train, y_test = train_test_split(X, y_noisy, test_size=0.25, random_state=42) 

    # Decide which values of lambda to use
    nlambdas = 100
    lambdas = np.logspace(-4, 10, nlambdas)

    #number of bootstrap samples
    #n_boostraps = 15 
    
    
    for n_bootstraps in [1, 20, 100]:
        print('polynomial degree:', degree,', number of bootstraps:', n_bootstraps)
        
        MSE_Bootstrap = np.zeros(nlambdas)
        bias = np.zeros(nlambdas)
        variance = np.zeros(nlambdas)

        for i in range(nlambdas):
            lmb = lambdas[i]
            model = Ridge(alpha = lmb)
            ypredict = np.empty((y_test.shape[0], n_bootstraps))    

            for j in range(n_bootstraps):
                x_, y_ = resample(X_train, y_train)

                #applying Linear Regression to the training data set
                ypredict[:, j] = model.fit(x_, y_).predict(X_test).ravel()       

            MSE_Bootstrap[i] = MSE(y_test, ypredict)
            
            if n_bootstraps==20 and degree==1:
                MSE_Bootstrap_compare_d1=MSE_Bootstrap
            elif n_bootstraps==20 and degree==6:
                MSE_Bootstrap_compare_d6=MSE_Bootstrap
            elif n_bootstraps==20 and degree==9:
                MSE_Bootstrap_compare_d9=MSE_Bootstrap
                
                
            bias[i] = np.mean( (y_test - np.mean(ypredict, axis=1, keepdims=True))**2 )
            variance[i] = np.mean( np.var(ypredict, axis=1, keepdims=True) )

        plt.plot(np.log10(lambdas), MSE_Bootstrap, label='MSE')
        plt.plot(np.log10(lambdas), bias, label='bias')
        plt.plot(np.log10(lambdas), variance, label='Variance')
        plt.xlabel('log10(lambda)')
        plt.ylabel('Prediction Error')
        plt.legend()
        plt.show()

In [ ]:
#Cross-validation on Ridge regression

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import resample
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

def MSE(y_data,y_model):
    n = np.size(y_model)
    return np.sum((y_data-y_model)**2)/n

#grid size
n = 200 

# Make data set.
x = np.linspace(0,(np.pi)/2,n)#.reshape(-1, 1)

#Northern H. parameters
s0 = 1
s2 = -0.473
a0 = 0.675
a2 = -0.192
i2 = -0.165

#flux function (eqn. (14) from Stone_1978)
y = 0.5*(s0*a2+s2*a0+(2/7)*s2*a2-i2)*((np.sin(x))**3-np.sin(x))

#noisy flux function
y_noisy = np.random.normal(y, abs(y*0.05))

for degree in [1, 6, 9]:
    poly = PolynomialFeatures(degree=degree)

    # Decide which values of lambda to use
    nlambdas = 100
    lambdas = np.logspace(-4, 10, nlambdas)

    for k in [5, 10, n]: #n for LOO C-V
        print('pol. degree=', degree, 'k=', k)
        kfold = KFold(n_splits = k)

        # Perform the cross-validation to estimate MSE
        MSE_KFold = np.zeros((nlambdas, k))

        c = 0
        for i in range(nlambdas):
            lmb = lambdas[i]
            model = Ridge(alpha = lmb)
            j = 0
            for train_inds, test_inds in kfold.split(x):
                xtrain = x[train_inds]
                ytrain = y_noisy[train_inds]

                xtest = x[test_inds]
                ytest = y_noisy[test_inds]

                Xtrain = poly.fit_transform(xtrain[:, np.newaxis])
                model.fit(Xtrain, ytrain[:, np.newaxis])

                Xtest = poly.fit_transform(xtest[:, np.newaxis])
                ypred = model.predict(Xtest)

                MSE_KFold[c,j] = MSE(ytest[:, np.newaxis], ypred)

                j += 1
            c += 1

        estimated_mse_KFold = np.mean(MSE_KFold, axis = 1)

## Cross-validation using cross_val_score from sklearn along with KFold

        estimated_mse_sklearn = np.zeros(nlambdas)

        d = 0
        for i in range(nlambdas):
            lmb = lambdas[i]
            model = Ridge(alpha = lmb)
            X = poly.fit_transform(x[:, np.newaxis])
            estimated_mse_folds = cross_val_score(model, X, y_noisy[:, np.newaxis], scoring='neg_mean_squared_error', cv=k)

            # cross_val_score return an array containing the estimated negative mse for every fold.
            # we have to the the mean of every array in order to get an estimate of the mse of the model
            estimated_mse_sklearn[d] = np.mean(-estimated_mse_folds)

            d += 1

        ## Plot    

        if k==5:          
            plt.figure()
            plt.plot(np.log10(lambdas), estimated_mse_sklearn, 'b', label = 'cross_val_score')
            plt.plot(np.log10(lambdas), estimated_mse_KFold, 'r--',  label = 'MSE KFold, k=5')
            if degree==1:
                plt.plot(np.log10(lambdas), MSE_Bootstrap_compare_d1, label='MSE Bootstrap')
            elif degree==6:
                plt.plot(np.log10(lambdas), MSE_Bootstrap_compare_d6, label='MSE Bootstrap')
            elif degree==9:
                plt.plot(np.log10(lambdas), MSE_Bootstrap_compare_d9, label='MSE Bootstrap')
                 
            plt.xlabel('log10(lambda)')
            plt.ylabel('MSE')
            plt.legend()
            plt.show()
        elif k==10:
            plt.figure()
            plt.plot(np.log10(lambdas), estimated_mse_sklearn, 'b', label = 'cross_val_score')
            plt.plot(np.log10(lambdas), estimated_mse_KFold, 'r--',  label = 'MSE KFold, k=10')
            if degree==1:
                plt.plot(np.log10(lambdas), MSE_Bootstrap_compare_d1, label='MSE Bootstrap')
            elif degree==6:
                plt.plot(np.log10(lambdas), MSE_Bootstrap_compare_d6, label='MSE Bootstrap')
            elif degree==9:
                plt.plot(np.log10(lambdas), MSE_Bootstrap_compare_d9, label='MSE Bootstrap')
            plt.xlabel('log10(lambda)')
            plt.ylabel('MSE')
            plt.legend()
            plt.show()
        else:
            plt.figure()
            plt.plot(np.log10(lambdas), estimated_mse_sklearn, 'b', label = 'cross_val_score')
            plt.plot(np.log10(lambdas), estimated_mse_KFold, 'r--',  label = 'MSE LOO')
            if degree==1:
                plt.plot(np.log10(lambdas), MSE_Bootstrap_compare_d1, label='MSE Bootstrap')
            elif degree==6:
                plt.plot(np.log10(lambdas), MSE_Bootstrap_compare_d6, label='MSE Bootstrap')
            elif degree==9:
                plt.plot(np.log10(lambdas), MSE_Bootstrap_compare_d9, label='MSE Bootstrap')
            plt.xlabel('log10(lambda)')
            plt.ylabel('MSE')
            plt.legend()
            plt.show()